In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [2]:
mnist = tf.keras.datasets.mnist
(X, y), (x_test, y_test) = mnist.load_data()

In [3]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.75)

In [46]:
model = keras.Sequential([
    layers.Reshape((28, 28, 1), input_shape=(1, 28, 28)), # reshape, conv2d expects 4 dims - batch size, height, width, and channels
    
    # Data Augmentation
    layers.RandomContrast(factor=0.1),
    layers.RandomWidth(factor=0.15), # horizontal stretch
    layers.RandomHeight(factor=0.15), # horizontal stretch

    # Block One
    layers.BatchNormalization(),
    layers.Conv2D(filters=64, kernel_size=4, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Block Two
    layers.BatchNormalization(),
    layers.Conv2D(filters=128, kernel_size=4, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Block Three
    layers.BatchNormalization(),
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Global Average Pooling to ensure the right size even though I stretch the images for data augmentation
    layers.GlobalAveragePooling2D(),

    # Head
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax'),
])

In [26]:
from tensorflow.keras.utils import to_categorical

X_train_reshaped = np.expand_dims(X_train, axis=1)
X_valid_reshaped = np.expand_dims(X_valid, axis=1)

# Check the shape of the reshaped data
print("Original shape:", X_train.shape)
print("Reshaped shape:", X_train_reshaped.shape)

# Convert labels to one-hot encoding
y_train_onehot = to_categorical(y_train, num_classes=10)
y_valid_onehot = to_categorical(y_valid, num_classes=10)

Original shape: (45000, 28, 28)
Reshaped shape: (45000, 1, 28, 28)


In [49]:
tf.random.set_seed(13)

optimizer = tf.keras.optimizers.Adam(epsilon=0.01)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy'],
)

history = model.fit(
    X_train_reshaped, y_train_onehot,
    validation_data=(X_valid_reshaped, y_valid_onehot),
    epochs=10,
)



Epoch 1/10
1407/1407 [==============================] - 120s 84ms/step - loss: 0.0188 - categorical_accuracy: 0.9940 - val_loss: 0.0347 - val_categorical_accuracy: 0.9911
Epoch 2/10
1407/1407 [==============================] - 120s 85ms/step - loss: 0.0200 - categorical_accuracy: 0.9940 - val_loss: 0.0466 - val_categorical_accuracy: 0.9869
Epoch 3/10
1407/1407 [==============================] - 123s 87ms/step - loss: 0.0193 - categorical_accuracy: 0.9938 - val_loss: 0.0387 - val_categorical_accuracy: 0.9896
Epoch 4/10
1407/1407 [==============================] - 114s 81ms/step - loss: 0.0191 - categorical_accuracy: 0.9938 - val_loss: 0.0296 - val_categorical_accuracy: 0.9923
Epoch 5/10
1407/1407 [==============================] - 113s 80ms/step - loss: 0.0151 - categorical_accuracy: 0.9953 - val_loss: 0.0696 - val_categorical_accuracy: 0.9790
Epoch 6/10
1407/1407 [==============================] - 111s 79ms/step - loss: 0.0149 - categorical_accuracy: 0.9952 - val_loss: 0.0375 - val_cat

In [61]:
model.save(filepath='digit_recognizer_v4.keras', overwrite=True)

In [54]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_6 (Reshape)         (None, 28, 28, 1)         0         
                                                                 
 random_contrast_6 (RandomC  (None, 28, 28, 1)         0         
 ontrast)                                                        
                                                                 
 random_width_3 (RandomWidt  (None, 28, None, 1)       0         
 h)                                                              
                                                                 
 random_height_2 (RandomHei  (None, None, None, 1)     0         
 ght)                                                            
                                                                 
 batch_normalization_22 (Ba  (None, None, None, 1)     4         
 tchNormalization)                                    

In [60]:
model.layers[5].get_weights()

[array([[[[ 0.11564885,  0.02600539,  0.09970675, ..., -0.01797443,
            0.02800392,  0.0551691 ]],
 
         [[ 0.11238336, -0.07653046, -0.06090117, ..., -0.01966076,
            0.04341546,  0.01843319]],
 
         [[ 0.01580799, -0.12005648, -0.09421268, ...,  0.06342409,
           -0.00283749, -0.15327306]],
 
         [[ 0.04153848, -0.01949162, -0.06791283, ...,  0.09489478,
           -0.18114552, -0.02198829]]],
 
 
        [[[ 0.08800013,  0.06704008,  0.11405012, ...,  0.0847474 ,
           -0.0343635 , -0.1424339 ]],
 
         [[ 0.00105619,  0.07189589,  0.01821665, ..., -0.13653323,
            0.03820055, -0.10021404]],
 
         [[ 0.00627409, -0.00620933,  0.04072127, ..., -0.03523374,
            0.10296713,  0.00581452]],
 
         [[ 0.00887623,  0.01513308, -0.0677743 , ...,  0.07849891,
            0.04091445, -0.03638228]]],
 
 
        [[[ 0.06463675,  0.00224594,  0.07448477, ..., -0.01204129,
           -0.12868401, -0.03332275]],
 
         [[-0

In [57]:
filters = model.layers[6].get_weights()[0]

plt.figure(figsize=(10, 10))
for i in range(filters.shape[-2]):
    plt.subplot(8, 8, i + 1)
    plt.imshow(filters[:, :, 0, i], cmap='gray')  # Assuming the filters are grayscale
    plt.axis('off')
plt.show()

IndexError: list index out of range